In [84]:
# Opzionale, scarica le sprite dei Pokémon per il gioco
!python ./scripts/sprite_downloader.py

Starting sprite download...
Not found (404): https://play.pokemonshowdown.com/sprites/ani/deoxysattack-forme.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani/wormadam-plant-cloak.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/deoxys-defense-forme.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani/deoxys-normal-forme.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/wormadam-trash-cloak.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani/wormadam-sandy-cloak.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/deoxys-speed-forme.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/wormadam-sandy-cloak.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/primal-groudon.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani/primal-kyogre.gif
Not found (404): https://play.pokemonshowdown.com/sprites/ani-back/deoxys-normal-forme.gif
Not 

In [55]:
# Librerie base
import pandas as pd
import numpy as np

# Machine Learning
import joblib
from sklearn.preprocessing import FunctionTransformer

# Management di file
import os

# Flask
from flask import Flask, request, jsonify, render_template

In [56]:
# Definizione dei percorsi e dei parametri globali
DATASET_BASE_PATH: str = './datasets'
COMBATS_PATH: str = f"{DATASET_BASE_PATH}/combats.csv"
TYPE_CHART_PATH: str = f"{DATASET_BASE_PATH}/type_chart.csv"
POKEMON_PATH: str = f"{DATASET_BASE_PATH}/pokemon.csv"
MODEL_DIR: str = './models'

In [57]:
def load_pokemon_data(combats_path: str, pokemon_path: str) -> pd.DataFrame:
   df_combats = pd.read_csv(combats_path)
   df_pokemon = pd.read_csv(pokemon_path, index_col="#").fillna("None")
   # Prerprocessing del campo winner per avere 0 se vince il primo Pokemon e 1 se vince il_So
   df_combats["Winner"] = (df_combats["Winner"] != df_combats["First_pokemon"]).astype(int)
   # Merge first and second Pokémon stats
   return df_combats.merge(
      df_pokemon.add_suffix("_F"), how="left",
      left_on="First_pokemon", right_index=True
   ).merge(
      df_pokemon.add_suffix("_S"), how="left",
      left_on="Second_pokemon", right_index=True
   ).drop(
      columns=["First_pokemon", "Second_pokemon"]
   )

In [58]:
# Caricamento dei DataFrame
df_type_chart = pd.read_csv(TYPE_CHART_PATH).fillna("None")
df_pokemon = pd.read_csv(POKEMON_PATH, index_col="#").fillna("None")
df_combats: pd.DataFrame = load_pokemon_data(COMBATS_PATH, POKEMON_PATH)

In [59]:
from utils import function_transformer
data_transformer: FunctionTransformer = FunctionTransformer(func=function_transformer, kw_args={'dataframe_type_chart': df_type_chart}).set_output(transform="pandas")

In [89]:
app: Flask = Flask(__name__, template_folder='webpage', static_folder="webpage/static")

models = os.listdir(MODEL_DIR)
model = joblib.load(os.path.join(MODEL_DIR, models[0]))

@app.route('/', methods=['GET'])
def one_v_one_battle():
    return render_template('1v1_battle.html', models=models, pokemon_choices=df_pokemon['Name'])

@app.route('/change_model', methods=['POST'])
def change_model():
    # Controlla se il corpo della richiesta è in formato JSON
    data = request.get_json()
    if not data:
        return jsonify({"error": "Invalid JSON."}), 400
    # Controlla se il modello specificato esiste
    model_filename: str = data.get('model_file')
    if not model_filename or model_filename not in models:
        return jsonify({"error": "Model file not found."}), 404
    # Cambia il modello di machine learning in uso
    global model
    model = joblib.load(os.path.join(MODEL_DIR, model_filename))
    return jsonify({"message": f"Model changed to {model_filename}."}), 200

@app.route('/precdict_battle', methods=['POST'])
def predict_battle():
    # Controlla se il corpo della richiesta è in formato JSON
    data = request.get_json()
    if not data:
        return jsonify({"error": "Invalid JSON."}), 400
    # Controlla se i Pokémon sono specificati
    p1: str = data.get("pokemon_first")
    p2: str = data.get("pokemon_second")
    if not p1 or not p2:
        return jsonify({"error": "Both Pokémon must be specified."}), 400
    # Prepara i dati per la previsione
    first_data = df_pokemon.loc[df_pokemon['Name'] == p1].add_suffix(suffix='_F')
    second_data = df_pokemon.loc[df_pokemon['Name'] == p2].add_suffix(suffix='_S')
    # Predizione del vincitore
    winner_id = model.predict(first_data.merge(second_data, how='cross'))
    winner = p1 if winner_id == 0 else p2
    confidence = np.max(model.predict_proba(first_data.merge(second_data, how='cross')), axis=1)[0] * 100
    return jsonify({
        "winner": winner,
        "confidence": float(confidence)
    })

app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [03/Jun/2025 11:20:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:20:06] "GET /static/img/sprites/Mega%20Venusaur_F.gif HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:20:06] "GET /static/img/sprites/Bulbasaur_S.gif HTTP/1.1" 304 -
127.0.0.1 - - [03/Jun/2025 11:20:07] "POST /precdict_battle HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:20:07] "GET /static/img/sprites/Mega%20Charizard%20X_S.gif HTTP/1.1" 304 -
127.0.0.1 - - [03/Jun/2025 11:20:08] "POST /precdict_battle HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:20:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:20:52] "GET /static/img/sprites/Mega%20Charizard%20X_S.gif HTTP/1.1" 304 -
127.0.0.1 - - [03/Jun/2025 11:20:52] "GET /static/img/sprites/Squirtle_F.gif HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2025 11:20:52] "POST /precdict_battle HTTP/1.1" 200 -
